# ACCESS THREDDS FOR HYCOM

Alli Ho 08/23/2021

Using Jerry's code `access_thredds.ipynb` to access HYCOM forecast from https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/GLBy0.08_930_FMRC_best.ncd.html

#### Adjusted code to work with HYCOM variables:
- `hycom_utils.py` was updated from `thredds_utils.py` to work with HYCOM output


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import numpy as np
import xarray as xr

import thredds_utils
import utils
from parcels_utils import xr_dataset_to_fieldset

INFO: Compiled ParcelsRandom ==> /var/folders/mb/hd5ymr3137g2g0chv_x_tvb40000gq/T/parcels-503/libparcels_random_42f48a59-05da-4e1e-b9e4-11130a30f4f4.so


## SETUP

Change the date and filename for when you want to start your deployment

Typically the forecast starts a week before today's date and runs for 300 hours (12.5 days), and so can forecast ~6 days ahead. 

Automatically set daterange to a week ahead:

In [3]:
# daterange = ("2021-08-24T00", "2021-09-03T00");
daterange = (np.datetime_as_string(np.datetime64('now')-np.timedelta64(1, 'D'), unit='h'), np.datetime_as_string(np.datetime64('now')+np.timedelta64(7, 'D'), unit='h'))
currentname = "hycom_mwbproj";

## DOWNLOAD

In [4]:
dataset_url = "https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/GLBy0.08_930_FMRC_best.ncd"

In [5]:
num_chunks = 1
# EXCLUDE THESE VARIABLES:
dropvars = {'water_temp', 'water_temp_bottom','salinity', 'salinity_bottom','water_u_bottom','water_v_bottom','surf_el'}

thredds_data = {
    thredds_utils.DATA_HYCOMFORE: xr.open_dataset(dataset_url, chunks={"time": 1},decode_times=False, drop_variables=dropvars),
}
t0 = np.datetime64(thredds_data[thredds_utils.DATA_HYCOMFORE].time.units[12:35])
tmp = thredds_data[thredds_utils.DATA_HYCOMFORE].time.data
# thredds_data[thredds_utils.DATA_HYCOMFORE]["timeday"] = t0 + np.array(tmp,dtype="timedelta64[h]")
thredds_data[thredds_utils.DATA_HYCOMFORE]["t0"] = np.timedelta64(t0 - np.datetime64('0000-01-01T00:00:00.000'), 'h')/np.timedelta64(1, 'D')
thredds_data[thredds_utils.DATA_HYCOMFORE] = thredds_data[thredds_utils.DATA_HYCOMFORE].assign_coords(time=(t0 + np.array(tmp,dtype="timedelta64[h]")))


thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].attrs["long_name"] = "Forecast time"
thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].attrs["standard_name"] = "time"
thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].attrs["_CoordinateAxisType"] = "Time"
thredds_data[thredds_utils.DATA_HYCOMFORE]["tau"].attrs["units"] = "hours since " + thredds_data[thredds_utils.DATA_HYCOMFORE]["tau"].time_origin

In [6]:
print("END OF FORECAST: "+ str(thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].data[-1]))
thredds_data[thredds_utils.DATA_HYCOMFORE];

END OF FORECAST: 2021-09-02T00:00:00.000000000


In [7]:
def get_region(data):
    time_range = get_time_slice(data[2])
    if data[6]:
        lat_range = utils.include_coord_range(data[3], thredds_data[data[1]]["lat"].values)
        lon_range = utils.include_coord_range(data[4], thredds_data[data[1]]["lon"].values)
    else:
        lat_range = data[3]
        lon_range = data[4]
        z_range = data[5]
    return dict(
        name = data[0],
        category = data[1],
        time = time_range,
        lat = lat_range,
        lon = lon_range,
        depth = z_range,
        domain = {
            "S": lat_range[0],
            "N": lat_range[1],
            "W": lon_range[0],
            "E": lon_range[1],
        }
    )

def get_latest_span(delta):
    # GMT, data recorded hourly
    time_now = np.datetime64("now", "h")
    return (time_now - delta, time_now)


def get_time_slice(time_range):
    if len(time_range) == 2:
        return slice(np.datetime64(time_range[0]), np.datetime64(time_range[1]))
    if len(time_range) == 3:
        # step size is an integer in hours
        return slice(np.datetime64(time_range[0]), np.datetime64(time_range[1]), time_range[2])
    

def get_regs_year(year, name, lat_rng, lon_rng):
    regions = []
    months = np.arange(str(year), str(year + 1), dtype="datetime64[M]")
    for m in months:
        days = np.arange(m, m + np.timedelta64(1, "M"), dtype="datetime64[D]")
        timerng = (np.datetime64(days[0], "h"), days[-1] + np.timedelta64(23, "h"))
        reg1 = (f"{name}_{m}", utils.DATA_1KM, timerng, lat_rng, lon_rng, False)
        reg2 = (f"{name}_{m}", utils.DATA_2KM, timerng, lat_rng, lon_rng, True)
        reg6 = (f"{name}_{m}", utils.DATA_6KM, timerng, lat_rng, lon_rng, True)
        regions.append(reg1)
        regions.append(reg2)
        regions.append(reg6)
    return regions

In [8]:
region_data = [
   (currentname, thredds_utils.DATA_HYCOMFORE, daterange, (22, 45), (273, 295), (0,0),False)
]

In [9]:
regions = []
for rd in region_data:
    new_reg = get_region(rd)
    if new_reg["time"].start >= np.datetime64("now", "h"):
        print(f"data starting from time {new_reg['time'].start} is in the future")
        continue
    new_reg["dataset"] = thredds_data[new_reg["category"]].sel(
        time=new_reg["time"],
        lat=slice(new_reg["lat"][0], new_reg["lat"][1]),
        lon=slice(new_reg["lon"][0], new_reg["lon"][1]),
        depth=slice(new_reg["depth"][0], new_reg["depth"][1]),
    )
    regions.append(new_reg)
    print(f"region {new_reg['name']} data megabytes: {new_reg['dataset'].nbytes / 1024 / 1024}")

region hycom_mwbproj data megabytes: 70.10061645507812


In [10]:
for r in regions:
    save_dir = utils.create_path(utils.CURRENT_NETCDF_DIR)
    filename = f"{r['name']}.nc"
    # save file
    r["dataset"].to_netcdf(save_dir / filename)
    print(f"saved to {save_dir / filename}")
print("DONE")

saved to current_netcdfs/hycom_mwbproj.nc
DONE
